## AutoML Tool - Helper Functions

# ==========================================================

This notebook will help automate required and mundane tasks of the model building process - data cleaning and feature engineering. The notebook assumes that some data exploration and data understanding has been done. The automated tasks conducted are as follows:

1. Drop columns with high % of missing values (threshold can be edited)
2. Drop numerical columns with no variance - all 1s
3. Drop categorical columns with all same labels - all 'Yes'
4. Drop categorical columns with too many labels (threshold can be edited)
5. Impute Missing Values for numerical and categorical data (filling technique can be set)
8. Transforms numerical values in a way that it will increase model accuracy

# ==========================================================

In [60]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

class Automl_tool():
    
    # funtion 1
    def drop_missing_columns(df, threshold):      
                    
        for i in df.columns:
            
            # calculating threshold
            counter = 0
            number_of_rows = df.shape[0]
            number_missing = sum(pd.isnull(df[i]))
            counter = number_missing/ (number_of_rows * 1.0)
            
            # dropping columns
            if counter > threshold:
                df.drop([i], axis = 1, inplace=True) 
            else:
                pass
        
        return df
    
        """
        Objective: Drops columns which has majority of rows missing
        
        Inputs:
        1. Dataframe df: Pandas dataframe
        2. threshold: Determines which columns will be dropped.
                      if threshold is .9, the columns with 90% missing value will be dropped
        
        Outputs:
        1. Dataframe df with dropped columns (if no columns are dropped, returns the same dataframe)
        """

    # funtion 2
    def drop_zero_variance_columns(df):
        
        # set numerical columns in variable
        numeric_cols = [var for var in df.columns if data[var].dtypes != 'O']
        
        # iterate over the columns and drop zero variance columns
        for i in numeric_cols:  
            if df[i].std() == 0.0:
                df.drop([i], axis = 1, inplace=True)
            else:
                pass
        
        return df
        
        """
        Objective: Drops numerical columns with zero variance
        
        Inputs:
        1. Dataframe df: Pandas dataframe
        
        Outputs:
        1. Dataframe df with dropped columns (if no columns are dropped, return the same dataframe)
        """
    
    # funtion 3
    def drop_zero_cardinality_columns(df):
        
        # set categorical columns
        categorical_cols = df.select_dtypes(include = ['object']).columns
        
        for i in categorical_cols:     
            if df[i].nunique() == 1:
                df.drop([i], axis = 1, inplace=True)
            else:
                pass
            
        return df
        
        """
        Objective: Drops categorical columns with same levels, such as a column with all 'yes' values
        
        Inputs:
        1. Dataframe df: Pandas dataframe
        
        Outputs:
        1. Dataframe df with dropped columns (if no columns are dropped, return the same dataframe)
        """        
     
    # funtion 4
    def drop_high_levels(df, threshold):
        
        categorical_cols = df.select_dtypes(include = ['object']).columns
        for i in categorical_cols:     
            if len(df[i].value_counts()) > threshold:
                df.drop([i], axis = 1, inplace=True)
            else:
                pass
            
        return df
                
        """
        this task will eliminate categorical columns if this column has a lot of levels. 
        inputs:
        1. Dataframe df: Pandas dataframe
        2. Threshold: How many levels you want at most
        
        outputs:
        1. Dataframe df: updated dataframe without dropped columns
        
        """
    
    # funtion 5
    def replace_missing(df, num_val):
        
        numeric_cols = [var for var in df.columns if data[var].dtypes != 'O']
        
        if num_val == 'mode':
            df[numeric_cols] = df[numeric_cols].fillna(value = df[numeric_cols].mode())
        
        elif num_val == 'mean':
            df[numeric_cols] = df[numeric_cols].fillna(value = df[numeric_cols].mean())

        elif num_val == 'median':
            df[numeric_cols] = df[numeric_cols].fillna(value = df[numeric_cols].median())

        else: # make 0
            df[numeric_cols] = df[numeric_cols].fillna(value = 0)
        
        # write unknown for categories
        df[df.select_dtypes(include = ['object']).columns.tolist()] = df[df.select_dtypes(include = ['object']).columns.tolist()].fillna(value = 'unknown')
        
        return df
    
        """
        Objective: Replaces missing values with given values
        
        Inputs:
        1. Dataframe df: Pandas dataframe
        2. num_val: User decides with what values they want to replace the missing numerical values. 
                    This value can be mean median mode or zero
        3. cat_val: User decides with what values they want to replace the missing categorical values. 
                    This value is 'unknown'
        
        Outputs:
        1. Dataframe df with imputed missing values
        """
    
    # funtion 6
    def transform(df, label_name):
        
        numeric_cols = [var for var in df.columns if data[var].dtypes != 'O']
        
        for i in numeric_cols:
        
            # initial dictionary
            corr = {'asis':0,'sqrt':0, 'log':0, 'pow2':0}

            # asis correlation
            corr['asis'] = abs(np.corrcoef(df[i], df[label_name])[1][0])

            #pow2 correlation
            corr['pow2'] = abs(np.corrcoef(np.power(df[i].subtract(df[i].mean())/df[i].std(), 2), df[label_name])[1][0])

            # log and sqrt
            if all((df[i]>=0)):
                corr['log'] = abs(np.corrcoef(np.log(df[i] + 0.00001), df[label_name])[1][0])
                corr['sqrt'] = abs(np.corrcoef(np.sqrt(df[i]), df[label_name])[1][0])
            else:
                corr['log'] = 0 
                corr['sqrt'] = 0 
            
            # select highest correlation 
            max_corr_type = max(corr, key=corr.get)

            if max_corr_type == 'sqrt':
                df[i] = np.sqrt(df[i]) # sqrt
            elif max_corr_type == 'pow2':
                df[i] = np.power(df[i],2) # power 2
            elif max_corr_type == 'log':
                df[i] = np.log(df[i] + 0.00001) # log      
            else:
                pass # for asis

        return df
        
        """
        Objective: Transforms numerical values in a way that it will increase model accuracy.
        try asis, sqrt, log, power(2) 
        inputs:
        1. Dataframe df: Pandas dataframe 
        
         outputs:
        1. Dataframe df with transformed values
        """

### Test Cases

In [58]:
data = pd.DataFrame({"A" : np.random.randint(low=1, high=100, size=10),
                     "B"  : np.random.normal(0.0, 1.0, size=10),
                     "C" : [1,2,3,4,5,6,7,8, np.nan,10]
                     })
data

,A,B,C
0,8,0.322218,1.0
1,74,0.077791,2.0
2,65,0.189813,3.0
3,78,0.436934,4.0
4,63,-0.554231,5.0
5,91,-0.661886,6.0
6,73,0.046093,7.0
7,65,-0.159302,8.0
8,85,-0.821768,NaN
9,50,-0.759525,10.0


In [63]:
# initialize class
automl = Automl_tool()

In [62]:
#Drop columns with high % of missing values (threshold can be edited)
automl.drop_missing_columns(df, threshold)

#Drop numerical columns with no variance - all 1s
automl.drop_zero_variance_columns(df)

#Drop categorical columns with all same labels - all 'Yes'
automl.drop_zero_cardinality_columns(df)

#Drop categorical columns with too many labels (threshold can be edited)
automl.drop_high_levels(df, threshold)

#Impute Missing Values for numerical and categorical data (filling technique can be set)
automl.replace_missing(df, num_val)

#Transforms numerical values in a way that it will increase model accuracy
automl.transform(df, label_name)

# =============================================================